In [19]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import matplotlib.pyplot as plt


df=pd.read_csv("./data_formatted/training_dataset.csv",low_memory=False)
df.match_date = df.match_date.apply(lambda x:datetime.strptime(x, '%Y-%m-%d'))

df = df.dropna()
df = df[(df.player1_plays > 10) & (df.player2_plays>10)]
df.shape

(35572, 36)

In [20]:
from sklearn.model_selection import train_test_split

X = df[["player1_name", "player1_age", "player1_atprank", "player1_plays", "player1_wins", "player1_losses", "player1_elo", "player1_mean_serve_rating", "player1_mean_atp_adversary", "player1_height", "player1_weight", "player1_oddsB365",
        "player2_name", "player2_age", "player2_atprank", "player2_plays", "player2_wins", "player2_losses", "player2_elo", "player2_mean_serve_rating", "player2_mean_atp_adversary", "player2_height", "player2_weight", "player2_oddsB365",
        "match_date","winner_player1"]]
#y = df.winner_player1
y = np.where((df.player1_oddsB365 > df.player2_oddsB365) & (df.winner_player1==1),1,0)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)# shuffle=False)


In [21]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler 

col_to_drop = ["player1_name", "player2_name", "match_date","winner_player1"]
X_train_filtered = X_train.drop(col_to_drop,axis=1)
X_test_filtered = X_test.drop(col_to_drop,axis=1)

scaler = MinMaxScaler()
scaler.fit(X_train_filtered[["player1_elo","player2_elo"]])
X_train_filtered[["player1_elo","player2_elo"]] = scaler.transform(X_train_filtered[["player1_elo","player2_elo"]])
X_test_filtered[["player1_elo","player2_elo"]] = scaler.transform(X_test_filtered[["player1_elo","player2_elo"]])

#scaler = StandardScaler()
#scaler.fit(X_train_filtered)
#X_train_filtered = scaler.transform(X_train_filtered)
#X_test_filtered = scaler.transform(X_test_filtered)

X_train_filtered = pd.DataFrame(X_train_filtered)
X_test_filtered = pd.DataFrame(X_test_filtered)

X_train_filtered.head()

player1_age  player1_atprank  player1_plays  player1_wins  \
39723    23.575633             22.0            127            71   
14036    34.184805             59.0            213           107   
33806    33.722108              2.0            922           791   
14371    30.573580             32.0            313           209   
14472    21.579740             14.0            148            80   

       player1_losses  player1_elo  player1_mean_serve_rating  \
39723              56     0.308011                 258.560963   
14036             106     0.175721                 248.840068   
33806             131     0.657305                 278.417755   
14371             104     0.304328                 264.708750   
14472              68     0.209484                 264.836977   

       player1_mean_atp_adversary  player1_height  player1_weight  ...  \
39723                   62.169769           185.0            84.0  ...   
14036                   60.869732           178.0            74.0  ...   
33806                   38.423510           185.0            85.0  ...   
14371                   68.719751           191.0            88.0  ...   
14472                   64.498897           196.0            91.0  ...   

       player2_atprank  player2_plays  player2_wins  player2_losses  \
39723             86.0             58            23              35   
14036             22.0            238           124             114   
33806             71.0            587           311             276   
14371             56.0            217           110             107   
14472             32.0            107            52              55   

       player2_elo  player2_mean_serve_rating  player2_mean_atp_adversary  \
39723     0.167189                 277.852632                   79.705590   
14036     0.238708                 253.224824                   66.041596   
33806     0.184598                 260.241466                   79.946392   
14371     0.216907                 248.809312                   60.499108   
14472     0.254078                 270.109890                   82.774650   

       player2_height  player2_weight  player2_oddsB365  
39723           191.0            87.0              3.25  
14036           183.0            73.0              1.57  
33806           185.0            78.0             10.00  
14371           178.0            74.0              2.62  
14472           183.0            86.0              1.66  

[5 rows x 22 columns]

In [22]:
import keras
from keras.layers import Dense
import matplotlib.pyplot as plt

model = keras.Sequential()
model.add(Dense(X_train_filtered.shape[1], activation='relu'))
model.add(Dense(X_train_filtered.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_focal_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_filtered, y_train,                        # Training dataset
                        epochs = 100,                         # Number of epochs
                        batch_size = 100,                     # Len of batch
                        validation_data=(X_test_filtered, y_test))    # Validation dataset

plt.figure(figsize=(12,4))

plt.subplot(121)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss by epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')

plt.subplot(122)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy by epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='right')
plt.show()

Epoch 1/100
285/285 [==============================] - 1s 1ms/step - loss: 3.3510 - accuracy: 0.7304 - val_loss: 0.6298 - val_accuracy: 0.8045
Epoch 2/100
285/285 [==============================] - 0s 810us/step - loss: 0.4707 - accuracy: 0.7487 - val_loss: 0.3436 - val_accuracy: 0.7768
Epoch 3/100
285/285 [==============================] - 0s 823us/step - loss: 0.3183 - accuracy: 0.7573 - val_loss: 0.2802 - val_accuracy: 0.8288
Epoch 4/100
285/285 [==============================] - 0s 794us/step - loss: 0.2584 - accuracy: 0.7661 - val_loss: 0.2740 - val_accuracy: 0.8458
Epoch 5/100
285/285 [==============================] - 0s 788us/step - loss: 0.2124 - accuracy: 0.7803 - val_loss: 0.1758 - val_accuracy: 0.7826
Epoch 6/100
285/285 [==============================] - 0s 839us/step - loss: 0.1908 - accuracy: 0.7809 - val_loss: 0.1698 - val_accuracy: 0.8448
Epoch 7/100
285/285 [==============================] - 0s 792us/step - loss: 0.1805 - accuracy: 0.7818 - val_loss: 0.1566 - val_accu

In [23]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(X_test_filtered)
print('Accuracy :', accuracy_score(y_predict.round(), y_test))

223/223 [==============================] - 0s 434us/step
Accuracy : 0.8400562192550949


In [28]:
df_roi = X_test.copy()
df_roi["y_predict_proba"] = y_predict   #all rows, first column = prediction score for winner 1 
df_roi["y_predict"] = y_predict.round()
df_roi["y_test"] = y_test

df_roi["proba_bookmaker"] = (1/df_roi.player1_oddsB365).round()
df_roi[["player1_name","player1_oddsB365","player2_name","player2_oddsB365", "winner_player1","proba_bookmaker","y_predict","y_predict_proba"]]


player1_name  player1_oddsB365          player2_name  \
18040    Kei Nishikori              2.25           Juan Monaco   
38319     David Goffin              1.20  Albert Ramos-Vinolas   
18243    Jeremy Chardy              1.53        Potito Starace   
26696        Dudi Sela              8.00          Gilles Simon   
26064   Lleyton Hewitt              3.75          Andy Roddick   
...                ...               ...                   ...   
45683     Daniel Evans              1.66       Karen Khachanov   
40777    Steve Johnson              1.72      Adrian Mannarino   
12603  Fabrice Santoro              3.50         Tomas Berdych   
28990      Marin Cilic              1.28         Pablo Andujar   
41432      Jiri Vesely              1.57  Nikoloz Basilashvili   

       player2_oddsB365  winner_player1  proba_bookmaker  y_predict  \
18040              1.57               1              0.0        0.0   
38319              4.50               1              1.0        0.0   
18243              2.37               1              1.0        0.0   
26696              1.08               0              0.0        0.0   
26064              1.25               1              0.0        0.0   
...                 ...             ...              ...        ...   
45683              2.10               1              1.0        0.0   
40777              2.00               1              1.0        0.0   
12603              1.28               0              0.0        0.0   
28990              3.50               0              1.0        0.0   
41432              2.37               1              1.0        0.0   

       y_predict_proba  
18040         0.467822  
38319         0.032481  
18243         0.309239  
26696         0.315573  
26064         0.355211  
...                ...  
45683         0.350615  
40777         0.365028  
12603         0.449178  
28990         0.097086  
41432         0.291508  

[7115 rows x 8 columns]

In [30]:
# ROI return of investment on all games
print("We bet 1€ on all test matches")
df_roi = X_test.copy()
df_roi["y_predict_proba"] = y_predict   #all rows, first column = prediction score for winner 1 
df_roi["y_predict"] = y_predict.round()
df_roi["y_test"] = y_test
df_roi["proba_bookmaker"] = (1/df_roi.player1_oddsB365).round()

#df_roi= df_roi.loc[df_roi["y_predict_proba"]>0.50]

money_invested=df_roi.shape[0]

money_won = 0.0
for index, row in df_roi.iterrows():
    if ((row.winner_player1 != row.proba_bookmaker) & (row.y_predict == 0)) :
        if row.winner_player1 == 0:
            money_won += row.player2_oddsB365
        else:
            money_won += row.player1_oddsB365

print("money spent:", money_invested,"€")
print("money won:",round(money_won,2),"€")
print("TOTAL:", round(money_won - money_invested, 2),"€")
print("ROI:", round(money_won*100/money_invested,2),"%")

We bet 1€ on all test matches
money spent: 7115 €
money won: 5997.79 €
TOTAL: -1117.21 €
ROI: 84.3 %
